In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import bitarray
#from pytorch_transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
#from transformers import CamembertLMHeadModel, CamembertTokenizer, CamembertForCausalLM, AutoTokenizer

import random

In [21]:
message_secret = "I dont want to be too cocky but here is a long ass message"
#message_secret = message_secret + '<eos>'

amorce = "I am going on a vacation to Italy. I am hoping that my"

model_name = "gpt2"

In [22]:
def limit_past(past):
    past = list(past)
    for i in range(len(past)):
        past[i] = past[i][:, :, :, -1022:]
    return past

def int2bits(inp, num_bits):
    if num_bits == 0:
        return []
    strlist = ('{0:0%db}' % num_bits).format(inp)
    return [int(strval) for strval in reversed(strlist)]

def num_same_from_beg(bits1, bits2):
    assert len(bits1) == len(bits2)
    for i in range(len(bits1)):
        if bits1[i] != bits2[i]:
            break

    return i

def bits2int(bits):
    res = 0
    for i, bit in enumerate(bits):
        res += bit * (2 ** i)
    return res


def str2bit(msg_str, tokenizer, model, context=None):
    if context is None:
        message_ctx = tokenizer.encode('<|endoftext|>')
    else: message_ctx = tokenizer.encode(context)
    #msg_str = "I am hidden"
    msg_str += '<eos>'
    msg_bits = bitarray.bitarray()
    msg_enc = decode_arithmetic(model, tokenizer, msg_str, message_ctx,
                                    precision=40, topk=60000, device='cpu')
    msg_bits = bitarray.bitarray(msg_enc)
        
    return msg_bits

def bit2str(msg_bits, tokenizer, model, context=None):
    if context is None:
        message_ctx = tokenizer.encode('<|endoftext|>')
    else: 
        message_ctx = tokenizer.encode(context)
        
    msg_str = encode_arithmetic(model, tokenizer, msg_bits, message_ctx,
        precision=40, topk=60000, device="cpu", model_device='cpu')
    print("msg str: ", msg_str, msg_str)
    msg_str = tokenizer.decode(msg_str)
    return msg_str

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = MeteorTokenizer.from_pretrained(model_name)
#tokenizer.unk_token = None
#tokenizer.bos_token = None
#tokenizer.eos_token = None

#model = GPT2LMHeadModel.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval();

In [24]:
encoded_message = tokenizer.encode(message_secret)
encoded_context = tokenizer.encode(amorce)

print(amorce)
print(encoded_context)

print(message_secret)
print(encoded_message)

tensor_amorce = torch.LongTensor(encoded_context).view(1,-1)
tensor_message = torch.LongTensor(encoded_message).view(1,-1)
#liste des tokens visibles sur:
#https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json

I am going on a vacation to Italy. I am hoping that my
[40, 716, 1016, 319, 257, 14600, 284, 8031, 13, 314, 716, 7725, 326, 616]
I dont want to be too cocky but here is a long ass message
[40, 17666, 765, 284, 307, 1165, 7540, 88, 475, 994, 318, 257, 890, 840, 3275]


In [25]:
tokens_temp = encoded_context.copy()
tensor_tokens = torch.LongTensor(tokens_temp).view(1,-1)
for i in range(15):
    with torch.no_grad():
            outputs = model(tensor_tokens)
            logits, past = outputs.logits, outputs.past_key_values
            
            tokens_temp.append(logits[0,-1].argsort()[-1].item())
            tensor_tokens = torch.LongTensor(tokens_temp).view(1,-1)
print(tokens_temp)
print(tokenizer.decode(tokens_temp))

[40, 716, 1016, 319, 257, 14600, 284, 8031, 13, 314, 716, 7725, 326, 616, 3656, 481, 307, 1498, 284, 1282, 736, 2582, 13, 314, 716, 1016, 284, 307, 736]
I am going on a vacation to Italy. I am hoping that my wife will be able to come back soon. I am going to be back


In [26]:
tokens_temp = encoded_context.copy()
tensor_tokens = torch.LongTensor(tokens_temp).view(1,-1)

for i in range(20):
    with torch.no_grad():
            outputs = model(tensor_tokens)
            logits, past = outputs.logits, outputs.past_key_values
            
            tokens_temp.append(logits[0,-1].argsort()[-round(random.random()*10)-1].item())
            tensor_tokens = torch.LongTensor(tokens_temp).view(1,-1)
print(tokens_temp)
print(tokenizer.decode(tokens_temp))

[40, 716, 1016, 319, 257, 14600, 284, 8031, 13, 314, 716, 7725, 326, 616, 1641, 11, 508, 389, 523, 1611, 11, 290, 284, 423, 884, 281, 6275, 9912, 13, 383, 1266, 6979, 373, 281]
I am going on a vacation to Italy. I am hoping that my family, who are so kind, and to have such an excellent holiday. The best gift was an


In [27]:
def decode_arithmetic(model, enc, text, context, device='cuda', temp=1.0, precision=16, topk=50000):
    # inp is a list of token indices
    # context is a list of token indices
    inp = enc.encode(text)
    # common BPE error case: 128, 128 (2 newlines) is interpretted as 628 (2 newlines)
    i = 0
    while i < len(inp):
        if inp[i] == 628:
            inp[i] = 198
            inp[i + 1:i + 1] = [198]
            i += 2
        else:
            i += 1

    context = torch.tensor(context[-1022:], device=device, dtype=torch.long)

    max_val = 2 ** precision
    threshold = 2 ** (-precision)
    cur_interval = [0, max_val]  # bottom inclusive, top exclusive

    prev = context
    past = None
    message = []
    with torch.no_grad():
        i = 0
        while i < len(inp):
            print(prev)
            outputs = model(prev.unsqueeze(0), past_key_values=past)
            past, logits = outputs.past_key_values, outputs.logits
            #if past is not None:
            #    past = limit_past(past)
            logits[0, -1, -1] = -1e10  # endoftext can't happen
            logits[0, -1, 628] = -1e10  # 2 newlines can't happen
            logits, indices = logits[0, -1, :].sort(descending=True)
            logits = logits.double()
            logits_temp = logits / temp
            probs_temp = F.softmax(logits_temp, dim=0)
            cum_probs = probs_temp.cumsum(0)

            # Cutoff low probabilities that would be rounded to 0
            cur_int_range = cur_interval[1] - cur_interval[0]
            cur_threshold = 1 / cur_int_range
            print(cur_threshold)
            
            k = min(max(2, (probs_temp < cur_threshold).nonzero()[0].item()), topk)
            probs_temp_int = probs_temp[:k]  # Cutoff all but top k

            # Rescale to correct range
            probs_temp_int = probs_temp_int / probs_temp_int.sum() * cur_int_range

            # Round probabilities to integers given precision
            probs_temp_int = probs_temp_int.round().long()
            cum_probs = probs_temp_int.cumsum(0)

            # Remove any elements from the bottom if rounding caused the total prob to be too large
            overfill_index = (cum_probs > cur_int_range).nonzero()
            if len(overfill_index) > 0:
                cum_probs = cum_probs[:overfill_index[0]]
                k = overfill_index[0].item()

            # Add any mass to the top if removing/rounding causes the total prob to be too small
            cum_probs += cur_int_range - cum_probs[-1]  # add

            # Covnert to position in range
            cum_probs += cur_interval[0]

            rank = (indices == inp[i]).nonzero().item()

            # Handle most errors that could happen because of BPE with heuristic
            if rank >= k:
                true_token_text = enc.decoder[inp[i]]
                for rank_idx in range(k):
                    prop_token_text = enc.decoder[indices[rank_idx].item()]
                    # common case that is not caught
                    if inp[i] == 128 and indices[rank_idx] == 198:
                        rank = rank_idx
                        inp[i] = indices[rank_idx].item()
                        break

                    # Is there a more likely prefix token that could be the actual token generated?
                    if len(prop_token_text) <= len(true_token_text) and \
                            prop_token_text == true_token_text[:len(prop_token_text)]:
                        rank = rank_idx
                        suffix = true_token_text[len(prop_token_text):]
                        suffix_tokens = enc.encode(suffix)  # a list
                        inp[i] = indices[rank_idx].item()
                        inp[i + 1:i + 1] = suffix_tokens  # insert suffix tokens into list
                        break

                    # Is there a more likely longer token that could be the actual token generated?
                    elif len(prop_token_text) > len(true_token_text) and \
                            true_token_text == prop_token_text[:len(true_token_text)]:
                        whole_text = true_token_text
                        num_extra = 1
                        while len(whole_text) < len(prop_token_text):
                            whole_text += enc.decoder[inp[i + num_extra]]
                            num_extra += 1
                        if prop_token_text == whole_text[:len(prop_token_text)]:
                            rank = rank_idx
                            inp[i] = indices[rank_idx].item()
                            for j in range(1, num_extra):
                                del inp[i + j]

                            if len(whole_text) > len(prop_token_text):
                                suffix = whole_text[len(prop_token_text):]
                                suffix_tokens = enc.encode(suffix)  # a list
                                inp[i + 1:i + 1] = suffix_tokens  # insert suffix tokens into list
                            break
                else:
                    print('Unable to fix BPE error: token received: %s=%d, text: %s' % (true_token_text, inp[i], text))
                    rank = 0

            selection = rank

            # Calculate new range as ints
            new_int_bottom = cum_probs[selection - 1] if selection > 0 else cur_interval[0]
            new_int_top = cum_probs[selection]

            # Convert range to bits
            new_int_bottom_bits_inc = list(reversed(int2bits(new_int_bottom, precision)))
            new_int_top_bits_inc = list(
                reversed(int2bits(new_int_top - 1, precision)))  # -1 here because upper bound is exclusive

            # Emit most significant bits which are now fixed and update interval
            num_bits_encoded = num_same_from_beg(new_int_bottom_bits_inc, new_int_top_bits_inc)
            if i == len(inp) - 1:
                new_bits = new_int_bottom_bits_inc
            else:
                new_bits = new_int_top_bits_inc[:num_bits_encoded]
            message += new_bits
            print(new_bits)

            #print("num bits: ",num_bits_encoded, "new bot 1: ", new_int_bottom_bits_inc[num_bits_encoded:])
            #print("new bot2 : ", [1] * num_bits_encoded)
            new_int_bottom_bits = new_int_bottom_bits_inc[num_bits_encoded:] + [0] * num_bits_encoded
            new_int_top_bits = new_int_top_bits_inc[num_bits_encoded:] + [1] * num_bits_encoded
            cur_interval[0] = bits2int(reversed(new_int_bottom_bits))
            cur_interval[1] = bits2int(reversed(new_int_top_bits)) + 1  # +1 here because upper bound is exclusive

            # Update history with new token
            prev = torch.tensor([inp[i]], device=device, dtype=torch.long)
            i += 1
            
    return message

print(message_secret)
bit_message = str2bit(message_secret, tokenizer, model)
print("encoded message: ", bit_message)

I dont want to be too cocky but here is a long ass message
tensor([50256])
9.094947017729282e-13
[0, 0, 1, 0]
tensor([40])
3.0938210371382067e-12
[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]
tensor([17666])
2.3179197916547297e-12
[1, 0, 0, 0]
tensor([765])
2.890773375613963e-12
[]
tensor([284])
3.954451855135105e-12
[0, 1]
tensor([307])
1.0748407960903596e-11
[0, 1, 1, 1, 0, 0, 0, 0, 1]
tensor([1165])
1.5807423970294914e-12
[1, 1, 0, 0, 0, 0, 0, 1, 0]
tensor([7540])
1.4325669280030835e-12
[]
tensor([88])
1.4530086583045491e-12
[0, 1]
tensor([475])
5.54258535083705e-12
[0, 1, 1, 1, 0, 1, 1]
tensor([994])
3.15972850394505e-12
[0]
tensor([318])
3.838655971446136e-12
[]
tensor([257])
1.5158692518647556e-11
[1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]
tensor([890])
1.7334601282257352e-12
[1, 0, 0, 1, 0, 1, 0, 0]
tensor([840])
6.06391349454872e-12
[0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1]
tensor([3275])
1.9324447033835973e-12
[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
tensor([27])
1.7725481207756337e-12

In [28]:
ah = np.array([[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [29]:
eh = [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [30]:
print(len(bit_message), len(eh))

167 108


In [31]:
def encode_arithmetic(model, enc, message, context, finish_sent=False, model_device="cuda", device='cpu', temp=1.0, precision=16,
                      topk=50000):
    context = torch.tensor(context[-1022:], device=device, dtype=torch.long)

    max_val = 2 ** precision
    threshold = 2 ** (-precision)
    cur_interval = [0, max_val]  # bottom inclusive, top exclusive

    prev = context
    output = context
    past = None

    total_num = 0
    total_num_for_stats = 0
    total_log_probs = 0
    total_kl = 0  # in bits
    total_entropy_ptau = 0
    total_num_sents = 0

    with torch.no_grad():
        i = 0
        sent_finish = False
        while i < len(message) or (finish_sent and not sent_finish):
            outputs = model(prev.unsqueeze(0).to(model_device), past_key_values=past)
            logits, past = outputs.logits, outputs.past_key_values
            logits = logits.to(device)
            
            logits[0, -1, -1] = -1e20  # endoftext token can't happen
            logits[0, -1, 628] = -1e20  # 2 newlines token can't happen
            logits, indices = logits[0, -1, :].sort(descending=True)
            logits = logits.double()
            logits_temp = logits / temp
            probs_temp = F.softmax(logits_temp, dim=0)
            log_probs_temp = F.log_softmax(logits_temp, dim=0)
            log_probs = F.log_softmax(logits, dim=0)
            print('z est reparti : ', probs_temp.cumsum(0)[:10], cur_interval[0], cur_interval[1])

            # conditions for having reached the end of the message
            if i >= len(message):
                selection = 0
                sent_finish = is_sent_finish(indices[selection].item(), enc)
            else:
                # Cutoff low probabilities that would be rounded to 0
                cur_int_range = cur_interval[1] - cur_interval[0]
                cur_threshold = 1 / cur_int_range
                k = min(max(2, (probs_temp < cur_threshold).nonzero()[0].item()), topk)
                probs_temp_int = probs_temp[:k]  # Cutoff all but top k

                # Rescale to correct range
                probs_temp_int = probs_temp_int / probs_temp_int.sum() * cur_int_range

                # Round probabilities to integers given precision
                probs_temp_int = probs_temp_int.round().long()
                cum_probs = probs_temp_int.cumsum(0)

                # Remove any elements from the bottom if rounding caused the total prob to be too large
                overfill_index = (cum_probs > cur_int_range).nonzero()
                if len(overfill_index) > 0:
                    cum_probs = cum_probs[:overfill_index[0]]

                # Add any mass to the top if removing/rounding causes the total prob to be too small
                cum_probs += cur_int_range - cum_probs[-1]  # add

                # Get out resulting probabilities
                probs_final = cum_probs.clone()
                probs_final[1:] = cum_probs[1:] - cum_probs[:-1]

                # Convert to position in range
                cum_probs += cur_interval[0]

                # Get selected index based on binary fraction from message bits
                message_bits = message[i:i + precision]
                if i + precision > len(message):
                    message_bits = message_bits + [0] * (i + precision - len(message))
                message_idx = bits2int(reversed(message_bits))
                selection = (cum_probs > message_idx).nonzero()[0].item()

                # Calculate new range as ints
                new_int_bottom = cum_probs[selection - 1] if selection > 0 else cur_interval[0]
                new_int_top = cum_probs[selection]

                # Convert range to bits
                new_int_bottom_bits_inc = list(reversed(int2bits(new_int_bottom, precision)))
                new_int_top_bits_inc = list(
                    reversed(int2bits(new_int_top - 1, precision)))  # -1 here because upper bound is exclusive
                # Consume most significant bits which are now fixed and update interval
                num_bits_encoded = num_same_from_beg(new_int_bottom_bits_inc, new_int_top_bits_inc)
                i += num_bits_encoded

                new_int_bottom_bits = new_int_bottom_bits_inc[num_bits_encoded:] + [0] * num_bits_encoded
                new_int_top_bits = new_int_top_bits_inc[num_bits_encoded:] + [1] * num_bits_encoded

                cur_interval[0] = bits2int(reversed(new_int_bottom_bits))
                cur_interval[1] = bits2int(reversed(new_int_top_bits)) + 1  # +1 here because upper bound is exclusive


            # Update history with new token
            prev = indices[selection].view(1)
            output = torch.cat((output, prev))
            total_num += 1
            
            print(indices[:10])

            # For text->bits->text
            partial = enc.decode(output[len(context):].tolist())
            print("==========================================")
            print("==========================================")
            if '<eos>' in partial:
                break

            print(selection, partial)

    return output[len(context):].tolist()



print(message_secret)
decoded_message = bit2str(bit_message, tokenizer, model)
print("decoded message: ", decoded_message)

I dont want to be too cocky but here is a long ass message
z est reparti :  tensor([0.0625, 0.1003, 0.1245, 0.1439, 0.1623, 0.1739, 0.1852, 0.1944, 0.2031,
        0.2109], dtype=torch.float64) 0 1099511627776
tensor([ 198,  464,    1,   32,   40,  818,   13, 1026,   50, 1212])
4 I
z est reparti :  tensor([0.0934, 0.1815, 0.2446, 0.2920, 0.3389, 0.3677, 0.3938, 0.4166, 0.4380,
        0.4543], dtype=torch.float64) 332922693888 656147595392
tensor([1101, 1053,  423,  716,  373,  836,  892,  550,  760,  460])
217 I dont
z est reparti :  tensor([0.3777, 0.5191, 0.5724, 0.6225, 0.6582, 0.6863, 0.7138, 0.7377, 0.7612,
        0.7787], dtype=torch.float64) 327169826816 758591139840
tensor([ 760,  892,  423,  765, 1107, 3505,  588, 1833,  766,  772])
3 I dont want
z est reparti :  tensor([0.7310, 0.7663, 0.7898, 0.8074, 0.8225, 0.8354, 0.8479, 0.8589, 0.8697,
        0.8787], dtype=torch.float64) 389766355392 735694544864
tensor([ 284,  345,  428,  262,  257,  616,  340,  661,  597, 2687])
0 

In [63]:

print(message_secret)
stegotexte = bit2str(bit_message, tokenizer, model, context=amorce)
print("stegotexte: ", stegotexte)

I am hidden
z est reparti :  tensor([0.0824, 0.1287, 0.1568, 0.1798, 0.2009, 0.2190, 0.2364, 0.2536, 0.2696,
        0.2825], dtype=torch.float64) 0 1099511627776
tensor([3656, 1641, 5229, 3367, 1545, 5296, 3397, 2460, 4957, 2988])
2  husband
z est reparti :  tensor([0.4606, 0.6631, 0.7136, 0.7574, 0.7925, 0.8078, 0.8184, 0.8289, 0.8377,
        0.8464], dtype=torch.float64) 64530923072 559138766160
tensor([ 481,  290,  460,   11,  318,  338,  743, 1839,  393,  468])
3  husband,
z est reparti :  tensor([0.1912, 0.2677, 0.3014, 0.3308, 0.3562, 0.3734, 0.3902, 0.4028, 0.4119,
        0.4207], dtype=torch.float64) 165110590752 859182163360
tensor([ 508,  616,  262,  290,  257,  314,  465, 2988, 4150,  355])
12  husband, wife
z est reparti :  tensor([0.4839, 0.9493, 0.9667, 0.9696, 0.9725, 0.9743, 0.9759, 0.9771, 0.9782,
        0.9790], dtype=torch.float64) 311723858368 683261009472
tensor([ 290,   11,  393, 1222,   12,  357,  481,   14, 3503,  286])
1  husband, wife,
z est reparti :  ten

In [64]:
bit_message_new = str2bit(stegotexte, tokenizer, model, context=amorce)
print("bit message: ", bit_message_new)

tensor([   40,   716,  1016,   319,   257, 14600,   284,  8031,    13,   314,
          716,  7725,   326,   616])
9.094947017729282e-13
[0, 0, 1, 0]
tensor([5229])
2.0218037663063934e-12
[0, 1, 1, 0, 0]
tensor([11])
1.4407736024146074e-12
[0, 1, 1, 0, 1, 1]
tensor([3656])
2.6915208802903317e-12
[]
tensor([11])
5.7838429002560465e-12
[1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1]
tensor([674])
2.207197579288877e-12
[]
tensor([4957])
2.257344000861651e-11
[0, 1, 1, 1, 1, 1, 1, 1, 1, 0]
tensor([357])
3.1225617738683937e-12
[1, 0, 0, 1, 1, 1, 1, 0, 1]
tensor([5832])
2.6590907208698316e-12
[1, 0, 0]
tensor([1839])
6.468492242599357e-11
[]
tensor([470])
6.494004150972388e-11
[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
tensor([588])
2.4171514166244248e-12
[0]
tensor([607])
6.4577819278929175e-12
[1, 0, 0, 0, 0, 0, 0, 0]
tensor([35751])
4.003089453523191e-12
[0, 1, 1, 1]
tensor([481])
1.61385168948465e-12
[1, 0, 0, 1]
tensor([423])
2.609029266595485e-12
[1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1]
tensor([28571])


In [65]:
print(message_secret)
decoded_message = bit2str(bit_message, tokenizer, model)
print("decoded message: ", decoded_message)

I am hidden
z est reparti :  tensor([0.0625, 0.1003, 0.1245, 0.1439, 0.1623, 0.1739, 0.1852, 0.1944, 0.2031,
        0.2109], dtype=torch.float64) 0 1099511627776
tensor([ 198,  464,    1,   32,   40,  818,   13, 1026,   50, 1212])
4 I
z est reparti :  tensor([0.0934, 0.1815, 0.2446, 0.2920, 0.3389, 0.3677, 0.3938, 0.4166, 0.4380,
        0.4543], dtype=torch.float64) 332922693888 656147595392
tensor([1101, 1053,  423,  716,  373,  836,  892,  550,  760,  460])
3 I am
z est reparti :  tensor([0.1394, 0.2198, 0.2520, 0.2804, 0.3054, 0.3279, 0.3497, 0.3712, 0.3910,
        0.4082], dtype=torch.float64) 548452273732 609675909336
tensor([ 257,  407, 1016,  281,  523, 3597,  262,  845, 3058, 1654])
7330 I am hidden
z est reparti :  tensor([0.2105, 0.3517, 0.4276, 0.4980, 0.5351, 0.5698, 0.6009, 0.6282, 0.6550,
        0.6795], dtype=torch.float64) 279390978048 554197581824
tensor([ 287,  422, 2157, 1497,  416,  739, 2641,  319,   11,   13])
2706 I am hidden<
z est reparti :  tensor([0.2927,

In [32]:
decoder = ArithmeticDecoder(model=model, tokenizer=tokenizer, topk=60000)
decoded_message = decoder.decode(encoded_message, context=encoded_context)
print("decoded message: ",decoded_message)
print(tokenizer.decode(decoded_message))

encoder = ArithmeticEncoder(model=model, tokenizer=tokenizer, topk=60000)
encoded_message = encoder.encode(message_secret, contexte=encoded_context)
print("encoded message: ",encoded_message)

decoder = ArithmeticDecoder(model=model, tokenizer=tokenizer, topk=60000)
decoded_message = decoder.decode(encoded_message)
print("decoded message: ",decoded_message)
print(tokenizer.decode(decoded_message))

NameError: name 'ArithmeticDecoder' is not defined

In [ ]:
print(amorce, message_secret)
#bit2str(encoded_message, model, tokenizer)[:-5]
#str2bits(model=model, enc=tokenizer, message=message_secret)
#decoded_message = bit2str(encoded_message, model, tokenizer)[:-5]
encoded_message = str2bits(model=model, enc=tokenizer, message=message_secret)
print(encoded_message)

In [ ]:
def is_sent_finish(token_idx, enc):
    token = enc.decode(token_idx)
    print(token)
    return '.' in token or '!' in token or '?' in token

def StegaGen(method, tokenizer, model, proc_seed): 
    
    if method == 'arithmetic':
        encoder_decoder = ArithmeticStega(tokenizer=tokenizer, model=model, seed=proc_seed) 
        
    elif method == 'adg':
        encoder_decoder = ArithmeticStega(tokenizer=tokenizer,model=model, seed=proc_seed)   
    else:
        print('Unkown method: ', method)
        
    return(encoder_decoder)


class ArithmetiEncoder:

    def __init__(self, **kwargs):
        self.tokenizer = tokenizer
        self.precision = kwargs.get("precision", 32)
        self.seed      = kwargs.get("seed", None)
        self.topk      = kwargs.get("topk", 10)
        self.topp      = kwargs.get("topp", None)
        self.model     = kwargs.get("model")
        
        self.g         = torch.Generator(device="cpu")
        if self.seed is None:
            self.g.seed()
        else:
            self.g.manual_seed(self.seed)
        pass
    
    def reset(self, context):
        return self.step(context, None)
    
    def reajust_probs(self, logits, indices):
        probs = F.softmax(logits , dim=0)
        if self.topp is not None:
            overfill_index = ((probs.cumsum(0) > self.topp).nonzero())[0]
            reajusted_probs = probs[:overfill_index]
            reajusted_indices = indices[:overfill_index]
        elif self.topk is not None:
            reajusted_probs = probs[:self.topk]
            reajusted_indices = indices[:self.topk]
        return(F.softmax(reajusted_probs , dim=0), reajusted_indices)
    
    def step(self, current, past):
        outputs = self.model(current.unsqueeze(0), past_key_values=past)
        logits, past_keys = outputs.logits, outputs.past_key_values
        
        logits[0, -1, -1] = -1e20  # endoftext token can't happen
        logits[0, -1, 628] = -1e20  # 2 newlines token can't happen
        logits, indices = logits[0, -1, :].sort(descending=True)
        
        probs, indices = self.reajust_probs(logits, indices)
        
        return(probs, past_keys, indices)
        
    def encode(self, private_message_bit: bitarray.bitarray, context: str = None):
        """
        :param msg: np array of 0s and 1s constituting a message bitstring
        :return:
        """
        message = private_message_bit
        tokenizer = self.tokenizer
        precision = self.precision

        max_val = 2 ** precision
        cur_interval = [0, max_val]  # bottom inclusive, top exclusive

        prev = context
        print("context is ", context)
        enc_prev = torch.LongTensor(tokenizer.encode(context))
        
        _, _, _ = self.reset(enc_prev)

        with torch.no_grad():
            i = 0
            j = 0
            past=None
            sent_finish = False
            while i < len(message) or (not sent_finish):

                probs, past, indices = self.step(enc_prev, past)
                          
                j += 1
                probs_temp = probs

                # conditions for having reached the end of the message
                if i >= len(message):
                    token = tokenizer.decode(token_idx)
                    sent_finish = '.' in token or '!' in token or '?' in token
                    print("token: ", token,"sent_finish: ",  sent_finish)
                else:
                    # Cutoff low probabilities that would be rounded to 0
                    cur_int_range = cur_interval[1] - cur_interval[0]

                    # Rescale to correct range
                    probs_temp_int = probs_temp * cur_int_range

                    # Round probabilities to integers given precision
                    probs_temp_int = probs_temp_int.round().long()
                    cum_probs = probs_temp_int.cumsum(0)
                    
                    # Remove any elements from the bottom if rounding caused the total prob to be too large
                    overfill_index = (cum_probs > cur_int_range).nonzero()
                    print(overfill_index, "uh")
                    if len(overfill_index) > 0:
                        cum_probs = cum_probs[:overfill_index[0]]

                    print(cum_probs[-1], "ah")
                    # Add any mass to the top if removing/rounding causes the total prob to be too small
                    cum_probs += cur_int_range - cum_probs[-1]  # add

                    # Get out resulting probabilities
                    probs_final = cum_probs.clone()
                    probs_final[1:] = cum_probs[1:] - cum_probs[:-1]

                    # Convert to position in range
                    cum_probs += cur_interval[0]

                    # Get selected index based on binary fraction from message bits
                    message_bits = message[i:i + precision]
                    if i + precision > len(message):
                        message_bits = message_bits + [0] * (i + precision - len(message))
                    message_idx = bits2int(reversed(message_bits))
                    selection = (cum_probs > message_idx).nonzero()[0].item()
                    
                    # Calculate new range as ints
                    new_int_bottom = cum_probs[selection - 1] if selection > 0 else cur_interval[0]
                    new_int_top = cum_probs[selection]

                    # Convert range to bits
                    new_int_bottom_bits_inc = list(reversed(int2bits(new_int_bottom, precision)))
                    new_int_top_bits_inc = list(reversed(int2bits(new_int_top - 1, precision)))  
                    # -1 here because upper bound is exclusive

                    # Consume most significant bits which are now fixed and update interval
                    num_bits_encoded = num_same_from_beg(new_int_bottom_bits_inc, new_int_top_bits_inc)
                    i += num_bits_encoded

                    new_int_bottom_bits = new_int_bottom_bits_inc[num_bits_encoded:] + [0] * num_bits_encoded
                    new_int_top_bits = new_int_top_bits_inc[num_bits_encoded:] + [1] * num_bits_encoded

                    cur_interval[0] = bits2int(reversed(new_int_bottom_bits))
                    cur_interval[1] = bits2int(reversed(new_int_top_bits)) + 1  
                    # +1 here because upper bound is exclusive

                    q = probs_final.double() / probs_final.sum()

                # Update history with new token
                prev = indices[selection].view(1)
                
                enc_prev = torch.cat((enc_prev, prev))
                print("current message: ", tokenizer.decode(enc_prev), prev, selection)

                # For text->bits->text
                partial = self.tokenizer.decode(enc_prev[len(context):].tolist())
                if '<eos>' in partial:
                    break


        return self.medium.enc.decode(enc_prev[len(context):].tolist()), enc_prev[len(context):].tolist()




In [ ]:
seed = -1
proc_seed = (seed * 55555) % 77777
proc_rng = np.random.default_rng(proc_seed)

# Now branch into different methods
encoder_decoder = StegaGen('arithmetic', tokenizer, model, proc_seed)

print(encoded_message, amorce)
ah = model( torch.LongTensor(tokenizer.encode(amorce)).unsqueeze(0)  )
logits=  ah.logits
print(logits.argmax())

# Proceed with communication
public_message_str, public_message_token = encoder_decoder.encode(private_message_bit=encoded_message,context=amorce)
print(public_message_str)

In [ ]:
    def decode(self, public_message_token, text=None, context: str = None, private_message_bitlen=None, **kwargs):

        inp = public_message_token
        enc = self.medium.enc
        device = self.medium.device  # may want to restrict to CPU!
        precision = self.precision
        max_val = 2 ** precision

        max_val = 2 ** precision
        cur_interval = [0, max_val]  # bottom inclusive, top exclusive
        
        t_iter_1 = None

        enc_context = self.medium.encode_context(context)
        enc_context = th.tensor(enc_context[-1022:], device=device, dtype=th.long)
        prev = enc_context
        message = []
        with th.no_grad():
            i = 0
            j = 0
            while i < len(inp):

                if t_iter_1 is not None:
                    delta_t_step_no_medium = time.time() - t_iter_1
                t_medium_1 = time.time()

                if j == 0:
                    probs, info = self.medium.reset(context=context)
                else:
                    probs, info = self.medium.step(prev)
                j += 1
                probs = th.from_numpy(probs.astype(np.float64))

                delta_t_medium = time.time() - t_medium_1
                t_iter_1 = time.time()

                probs_temp = probs
                indices = th.from_numpy(info["indices"])
                probs_temp_int = probs_temp

                # Cutoff low probabilities that would be rounded to 0
                cur_int_range = cur_interval[1] - cur_interval[0]

                # Rescale to correct range
                probs_temp_int = probs_temp_int / probs_temp_int.sum() * cur_int_range

                # Round probabilities to integers given precision
                probs_temp_int = probs_temp_int.round().long()
                cum_probs = probs_temp_int.cumsum(0)

                # Remove any elements from the bottom if rounding caused the total prob to be too large
                overfill_index = (cum_probs > cur_int_range).nonzero()
                if len(overfill_index) > 0:
                    cum_probs = cum_probs[:overfill_index[0]]
                    k = overfill_index[0].item()

                # Add any mass to the top if removing/rounding causes the total prob to be too small
                cum_probs += cur_int_range - cum_probs[-1]  # add

                # Covnert to position in range
                cum_probs += cur_interval[0]

                rank = (indices == inp[i]).nonzero().item()

                k = len(probs)
                # Handle most errors that could happen because of BPE with heuristic
                if rank >= k:
                    true_token_text = enc.decoder[inp[i]]
                    for rank_idx in range(k):
                        prop_token_text = enc.decoder[indices[rank_idx].item()]
                        # common case that is not caught
                        if inp[i] == 128 and indices[rank_idx] == 198:
                            rank = rank_idx
                            inp[i] = indices[rank_idx].item()
                            break

                        # Is there a more likely prefix token that could be the actual token generated?
                        if len(prop_token_text) <= len(true_token_text) and \
                                prop_token_text == true_token_text[:len(prop_token_text)]:
                            rank = rank_idx
                            suffix = true_token_text[len(prop_token_text):]
                            suffix_tokens = enc.encode(suffix)  # a list
                            inp[i] = indices[rank_idx].item()
                            inp[i + 1:i + 1] = suffix_tokens  # insert suffix tokens into list
                            break

                        # Is there a more likely longer token that could be the actual token generated?
                        elif len(prop_token_text) > len(true_token_text) and \
                                true_token_text == prop_token_text[:len(true_token_text)]:
                            whole_text = true_token_text
                            num_extra = 1
                            while len(whole_text) < len(prop_token_text):
                                whole_text += enc.decoder[inp[i + num_extra]]
                                num_extra += 1
                            if prop_token_text == whole_text[:len(prop_token_text)]:
                                rank = rank_idx
                                inp[i] = indices[rank_idx].item()
                                for j in range(1, num_extra):
                                    del inp[i + j]

                                if len(whole_text) > len(prop_token_text):
                                    suffix = whole_text[len(prop_token_text):]
                                    suffix_tokens = enc.encode(suffix)  # a list
                                    inp[i + 1:i + 1] = suffix_tokens  # insert suffix tokens into list
                                break
                    else:
                        print('Unable to fix BPE error: token received: %s=%d, text: %s' % (
                            true_token_text, inp[i], text))
                        rank = 0

                selection = rank

                # Calculate new range as ints
                new_int_bottom = cum_probs[selection - 1] if selection > 0 else cur_interval[0]
                new_int_top = cum_probs[selection]

                # Convert range to bits
                new_int_bottom_bits_inc = list(reversed(int2bits(new_int_bottom, precision)))
                new_int_top_bits_inc = list(reversed(int2bits(new_int_top - 1, precision)))  
                # -1 because upper bound is exclusive

                # Emit most significant bits which are now fixed and update interval
                num_bits_encoded = num_same_from_beg(new_int_bottom_bits_inc, new_int_top_bits_inc)
                if i == len(inp) - 1:
                    new_bits = new_int_bottom_bits_inc
                else:
                    new_bits = new_int_top_bits_inc[:num_bits_encoded]
                message += new_bits

                new_int_bottom_bits = new_int_bottom_bits_inc[num_bits_encoded:] + [0] * num_bits_encoded
                new_int_top_bits = new_int_top_bits_inc[num_bits_encoded:] + [1] * num_bits_encoded

                cur_interval[0] = bits2int(reversed(new_int_bottom_bits))
                cur_interval[1] = bits2int(reversed(new_int_top_bits)) + 1  
                # +1 because upper bound is exclusive

                # Update history with new token
                prev = th.tensor([inp[i]], device=device, dtype=th.long)
                i += 1

        output = bitarray.bitarray(message)
        if private_message_bitlen is not None:
            output = output[:private_message_bitlen]
        return output
